<a href="https://colab.research.google.com/github/ZacharySBrown/dapt-615/blob/master/modules/0_acc_dist_python/cudf_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install `RAPIDS` and Download Data

In [1]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

!pip install -U cffi

#!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

#import sys, os

#dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
#sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
#sys.path
#exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

!wget https://vcu-dapt-615.s3.amazonaws.com/covid_19_data.csv

Requirement already up-to-date: cffi in /usr/local/lib/python3.7/dist-packages (1.14.5)
Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 12.31 MiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cud

# Understanding and Applying Data Analysis with Python


## Outline

1. File I/O and working with data in pandas

    * Reading, writing, and creating structured data in Python
    * Viewing, inspecting, and selecting data
    
    
2. Exploratory data analysis with pandas

    * Data cleaning
    * Summary statistics
    * Data transformations
    * Sorting, aggregation, joins and pivots
    * Data visualization

## Project Jupyter

Project Jupyter exists to develop open-source software, open-standards, and services for interactive computing across dozens of programming languages. Jupyter supports over 40 programming languages, including Python, R, Julia, and Scala.

The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, data visualization, machine learning, and much more.

Try Jupyter without installing anything: https://jupyter.org/try

## Novel Corona Virus 2019 Dataset: day level information on COVID-19 affected cases

### About this dataset:

https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset/data?select=covid_19_data.csv

(original: Johns Hopkins Github repository: https://github.com/CSSEGISandData/COVID-19)

Name of the file: "covid_19_data.csv"

Columns description:

- **Sno** - Serial number
- **ObservationDate** - Date of the observation in MM/DD/YYYY
- **Province/State** - Province or state of the observation (Could be empty when missing)
- **Country/Region** - Country of observation
- **Last Update** - Time in UTC at which the row is updated for the given province or country. (Not standardised and so please clean before using it)
- **Confirmed** - Cumulative number of confirmed cases till that date
- **Deaths** - Cumulative number of of deaths till that date
- **Recovered** - Cumulative number of recovered cases till that date

## 1. File I/O and working with data in pandas

### Reading Data

In [2]:
#To start working with cuDF, we need to import this library:
import cudf

#We'll also import the Pandas library as well for occassional performance comparisons:
import pandas as pd

Most of the time, you will be probably working with data that already exists in variety of different formats.
By far the most basic of these are CSV or Excel files. This is how you read these files with pandas:

In [3]:
#read CSV file in pandas:
pd_covid = pd.read_csv('covid_19_data.csv')

covid = cudf.from_pandas(pd_covid)

Let's pause here for a minute, and try to understand what exactly happens once we run **pandas.read_csv** command.

Two core objects in pandas are the **DataFrame** and the **Series**.

A **DataFrame** is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row and a column. The list of row labels used in a DataFrame is known as an Index.

In our example above, *covid* is a DataFrame created from 'covid_19_data.csv' file. 


In [15]:
#We can examine the contents of our DataFrame using the head() command:
covid.head()

,SNo,ObservationDate,ProvinceState,CountryRegion,LastUpdate,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


From this example, we can see that DataFrame entries are not limited to integers. For instance, there is an entry with value of "Beijing", which is a string.

A **Series**, the second core pandas object, is a sequence of data values. If a DataFrame is a table, a Series is a list and,  in essence, it is a single column of a DataFrame.

In [16]:
#This is how we can see all the Series names in our DataFrame:
covid.columns

Index(['SNo', 'ObservationDate', 'ProvinceState', 'CountryRegion',
       'LastUpdate', 'Confirmed', 'Deaths', 'Recovered'],
      dtype='object')

## Viewing, inspecting, and selecting data

Let's return to our **covid** DataFrame. Remember - you can use *head()* method to view the first *n* rows:

In [17]:
#View first 3 rows of data set:
covid.head(3)

,SNo,ObservationDate,ProvinceState,CountryRegion,LastUpdate,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0


In [18]:
#In a similar way, you can inspect last n rows with tail() method:
covid.tail()

,SNo,ObservationDate,ProvinceState,CountryRegion,LastUpdate,Confirmed,Deaths,Recovered
172475,172476,12/06/2020,Zaporizhia Oblast,Ukraine,2020-12-07 05:26:14,36539.0,337.0,6556.0
172476,172477,12/06/2020,Zeeland,Netherlands,2020-12-07 05:26:14,6710.0,104.0,0.0
172477,172478,12/06/2020,Zhejiang,Mainland China,2020-12-07 05:26:14,1295.0,1.0,1288.0
172478,172479,12/06/2020,Zhytomyr Oblast,Ukraine,2020-12-07 05:26:14,31967.0,531.0,22263.0
172479,172480,12/06/2020,Zuid-Holland,Netherlands,2020-12-07 05:26:14,154813.0,2414.0,0.0


Pandas objects have a number of attributes enabling you to access the metadata. We can also use the **shape** attribute to check how large this DataFrame is:

In [19]:
#Check the data set size:
covid.shape

(172480, 8)

To see list of all columns, use *columns* attribute:

In [20]:
#See the list of all DataFrame columns:
covid.columns

Index(['SNo', 'ObservationDate', 'ProvinceState', 'CountryRegion',
       'LastUpdate', 'Confirmed', 'Deaths', 'Recovered'],
      dtype='object')

There are two ways to access values in any column within the DataFrame. One is the following:

In [21]:
#Look at first 5 rows of 'ObservationDate' column:
covid.ObservationDate.head()

0    01/22/2020
1    01/22/2020
2    01/22/2020
3    01/22/2020
4    01/22/2020
Name: ObservationDate, dtype: object

And another way is to use Python indexing (**[ ]**) operator:

In [4]:
#Look at first 5 rows of 'Last Update' column:
covid['LastUpdate'].head()

0    1/22/2020 17:00
1    1/22/2020 17:00
2    1/22/2020 17:00
3    1/22/2020 17:00
4    1/22/2020 17:00
Name: LastUpdate, dtype: object

(The advantage of this way is that it handles column names with reserved characters in them.)

We can also select multiple columns by providing list of their names (this will return columns as a new DataFrame):

In [5]:
#select multiple columns:
covid[['ObservationDate', 'Confirmed', 'Recovered']].head()

,ObservationDate,Confirmed,Recovered
0,01/22/2020,1.0,0.0
1,01/22/2020,14.0,0.0
2,01/22/2020,6.0,0.0
3,01/22/2020,1.0,0.0
4,01/22/2020,0.0,0.0


To look at a single specific value within Series, we need to use the indexing operator once more:

In [7]:

!pip install -U cffi

     |████████████████████████████████| 402 kB 15.7 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.14.3
    Uninstalling cffi-1.14.3:
      Successfully uninstalled cffi-1.14.3


In [8]:
covid['ProvinceState'][3]

Exception: ignored

In [32]:
covid.head()

,SNo,ObservationDate,ProvinceState,CountryRegion,LastUpdate,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In addition to Python-native indexing operator, we can also use **iloc** and **loc**. 

We use **iloc** to select data based on its numerical position. 

In [33]:
#Select first row of data:
covid.iloc[0]

Exception: ignored

In [ ]:
#If, instead, we want to get first column of data, we would use:
covid.iloc[:, 2].head()

We use **loc** for label-based selection:

In [ ]:
covid.loc[:, ['ObservationDate', 'Country/Region', 'Confirmed']].head()

Usually, data selection based on *conditions* provides the most interesting insights.

Let's take a look at Italy-related data:

In [ ]:
#Select 'Italy' Country/Region data:
covid[covid['Country/Region'] == 'Italy']

To see the number of confirmed cases in Italy as of April 13, we can use the apmersand (**&**) to bring the two questions together:

In [ ]:
#Confirmed cases in Italy as of April 13, 2020:
covid[(covid['Country/Region'] == 'Italy') & (covid['ObservationDate'] == '04/13/2020')].Confirmed

If, instead, we are interested in seeing all the data related to California or Washington states in US, we would use a pipe (**|**):

In [ ]:
covid[(covid['Country/Region'] == 'US') &
      ((covid['Province/State'] == 'Washington') | (covid['Province/State'] == 'California'))].head(7)

To get the same results, we might also use pandas built-in conditional selector **isin()**:

In [ ]:
#Alternative way of looking at all data relates to CA and WA states in US:
covid[(covid['Country/Region'] == 'US') &
      (covid['Province/State'].isin(['Washington', 'California']))].head(7)

### Assigning data

Sometimes we might want to re-write all the values within the Series. For example, we can create a new column "Source", and fill it in with "Trusted":

In [ ]:
#Create a new "Source" column:
covid['Source'] = 'Trusted'
covid.head()

## 2. Exploratory data analysis with pandas

### Data cleaning

#### Remove unnecessary data

Now that we have covered some basics of data exporting and viewing, let's see how we can clean our sample dataset for further analysis.

First of all, let's delete the "Source" column we just created. To do this, we will use **drop()** method. Its *axis* parameter identifies whether we want to drop labels from the index (0 or ‘index’) or columns (1 or ‘columns’):

In [ ]:
#drop "Source', 'SNo', and 'LastUpdate' columns:
covid = covid.drop(['Source', 'Last Update', 'SNo'], axis = 1)
covid.head(2)

(Here, we also dropped 'SNo' column, which is basically another index column, and 'Last Update' column since it was showing time in UTC at which the row is updated for the given province or country.)

In [ ]:
#To drop a row (or multiple rows) by index, we would use the following syntax:
#covid.drop([0, 1]) - drop first two rows

#### Rename columns

Sometimes, data sets we are working with come with column names or index names which are not very convenient. For example, we have 'Province/State' and 'Country/Region' columns which have special characters in their names, and also 'Last Update' column with a space in it. Let's rename them:

In [ ]:
#Rename 'Province/State' column to a 'Region', and 'Country/Region' column to a 'Country':
covid = covid.rename(columns = {'Province/State': 'Region', 'Country/Region': 'Country'})
covid.head()

#### Remove duplicate records

First, we check if there are any duplicate records using **duplicated()** method which returns boolean True/False values:

In [ ]:
# NOT SUPPORTED IN RAPIDS
covid.duplicated().head()

Of course we do not want to scrool through thousands of returned True/False. Instead, we can check for unique vaules:

In [ ]:
#Check for duplicated values using unique() method:
covid.drop_duplicates().shape

In [ ]:
covid.shape

To remove duplicated records from the DataFrame, we would use the following syntax:

**df = df.drop_duplicates()**

To remove duplicates of only one or a subset of columns, we can specify *subset* as the individual column or list of columns that should be unique:

**df.drop_duplicates(subset = ['Column_1', 'Column_2'], keep = 'False')**

We can do the same task conditional on a different column’s value. In such case we can **sort_values()** first, and specify **keep** equals either first or last:

**df.sort_values('Column_1', ascending=False)**

**df = df.drop_duplicates(subset='Column_2', keep='first')**

#### Data types

We should also investigate data types within our dataframe. To do this, we use **dtype** (for a Series) or **dtypes** (for a DataFrame) attribute:

In [ ]:
#check all data types within our dataset:
covid.dtypes

In [ ]:
#check data type of specific column:
covid.Confirmed.dtype

Note that columns consisting entirely of strings do not get their own type in pandas; they are instead given the object type.

Sometimes we would like to convert a column of one type into another wherever such a conversion makes sense by using the **astype()** function. In our dataframe, we may transform the *Deaths* column from its existing float64 data type into a int64 data type:

In [ ]:
#Convert 'Deaths' values to integers:
covid.Deaths = covid.Deaths.fillna(0).astype('int64')
covid.dtypes

Also, to make work with dates easier, let's convert 'ObservationDate' column to pandas *datetime* object:

In [ ]:
#Convert ObservationDate column to datetime:
covid['ObservationDate'] = cudf.to_datetime(covid.ObservationDate)

In [ ]:
covid.dtypes

#### Convert strings to uppercase:

Many times, when we work with string data in our data set, it might be a good idea to convert all characters to uppercase and strip extra whitespaces before and after each string.

In [ ]:
covid.head(2)

Let's modify strings within the 'Region' and 'Country' Series of our data set. To convert strings in the Series/Index to uppercase, we can use **upper()** method:

In [ ]:
#Convert 'Region' and 'Country' to uppercase:
covid.Region = covid.Region.str.upper()
covid.Country = covid.Country.str.upper()

In [ ]:
covid.head()

To remove leading and trailing characters, we would use **strip()** method:

In [ ]:
covid.Region = covid.Region.str.strip()
covid.Country = covid.Country.str.strip()

Note: Sometimes, we might want to remove leading (**lstrip()**) or trailing (**rstrip()**) characters only.

#### Missing values

Usually, when we work with real-world data, we always have some missing records. To find these missing values, also known as **NaN** (Not a Number), in our data set, we would use **isnull** method (or, alternatively, **notnull()** method which will select not empty values)

In [ ]:
#Check which coumns in our data set have missing values:
covid.isnull().any()

In [ ]:
covid[covid.Region.isnull()].head()

In [ ]:
#Check how many values are missing:
percent_missing = covid.isnull().sum() * 100 / len(covid)
missing_df = cudf.DataFrame({'column_name': covid.columns,
                                 'percent_missing': percent_missing})
missing_df

To perform data analysis, we would like to replace missing values, and pandas provides a really handy method for this problem: **fillna()**. 

In our case, we can simply replace each NaN with an 'Unknown':

In [ ]:
#To drop all the missing values in one column:
#df = df.dropna(subset = ['Column_Name'])

In [ ]:
#Replace missing 'Region' calues with 'Unknown':
covid['Region'] = covid['Region'].fillna('Unknown')
covid.iloc[35:40]

In [ ]:
covid.isnull().any()

Sometimes it makes sense to remove missing values from data set, and in that case we would use **dropna()** method. By default, **dropna()** will drop all rows in which any null value is present:

**df.dropna()**

We can also drop missing values along a different axis; *axis=1* drops all columns containing a null value.

### Summary statistics

Pandas provides some handy methods which allow us to see a high-level summary of data. We already familiar with one of these methods - **describe()**:

In [ ]:
#Applied to numerical data:
covid.Confirmed.describe()

In [ ]:
covid.head()

In [ ]:
#Applied to string data:
# NOT SUPPORTED IN RAPIDS!!!
covid.Country.describe()


For the reference, here are some others commonly used methods (they can be applies to a Series as well):

**df.describe()** - summary statistics for numerical columns

**df.mean()** - returns the mean of all columns

**df.corr()** - returns the correlation between columns in a DataFrame

**df.count()** - returns the number of non-null values in each DataFrame column

**df.max()** - returns the highest value in each column

**df.min()** - returns the lowest value in each column

**df.median()** - returns the median of each column

**df.std()** - returns the standard deviation of each column


To see a list of all unique values in certain Series, we would use the **unique()** method:

In [ ]:
#See the list of unique 'Country' values:
covid.Country.unique()

**nunique()** will return a number of unique values:

In [ ]:
#Number of unique countries:
covid.Country.nunique()

### Data transformations with 'map' and 'apply'

There are two methods in pandas which allow us to take one set of values and "map" them to another set. **map()** returns a new Series where all the values have been transformed.

Let's say we have noticed that 'Country' Series contains, among others, entries for 'Mainland China', 'Hong Kong', and 'Macau'. We know that they are all parts of People's Republic of China, and would prefer to use 'China' for all of them. Here is how this can be done:

In [ ]:
s = set(['MAINLAND CHINA', 'HONG KONG', 'MACAU'])

country_pandas = covid['Country'].to_pandas()

country_pandas_fixed = country_pandas.map(lambda x: 'CHINA' if x in s else x)
country_pandas_fixed.head()
covid['GeneralCountry'] = country_pandas_fixed
covid.head()

Let's say we needed to adjust our confirmed up by a single case for every observation. For this, we can use the `applymap` method on a single _numeric_ `cudf` column (`Series`) in the `DataFrame`. 

In [ ]:
covid['Confirmed_Adjusted'] = covid['Confirmed'].applymap(lambda x: x + 1.0)
covid.head()

Here we used a small anonymous function (*lambda function*) which can take any number of arguments, but can only have one expression. Once the expression is executed, it returns the result.


**apply_rows()** is a similar method which be applied to DataFrames. The difference is that **apply_rows()** works on a row/column basis of a DataFrame, while **map()** works element-wise on a Series.


Let's create a function which calculates "naive" mortality rate, and write its output in a new 'MortalityRate' column using **apply_rows()** method:

In [ ]:
#Create MortalityRate column using apply() method:
import numpy as np
def naive_rate(deaths, confirmed, MortalityRate):
    for it, (d, c) in enumerate(zip(deaths, confirmed)):
        if c > 0:
            MortalityRate[it] = d/c
        else:
            MortalityRate[it] = 0
            
covid = covid.apply_rows(naive_rate,
                   incols={'Deaths':'deaths', 'Confirmed':'confirmed'},
                   outcols={'MortalityRate': np.float64},
                   kwargs={}
                  )
covid[covid.MortalityRate != 0].head()

### Sorting 

In many cases, we would like to first sort values in our data set, and then perform further data manipulations. To sort data and get it in the desired order we can use **sort_values()** method.

In [ ]:
#Sort values by 'GeneralCountry':
covid.sort_values(by = 'GeneralCountry').head()

**sort_values()** defaults to an ascending sort, where the lowest values go first. To obtain a descending sort, we need to specify *ascending* parameter: 

In [ ]:
#Sort values by 'ObservationDate' and 'Confirmed' in descending order:
covid.sort_values(by = ['ObservationDate', 'Confirmed'], ascending = False).head(2)

### Grouping

Depending on our goals, we might prefer to group data first, and then perform some operations on this group. In pandas, we can do it with **groupby()** method. However, simple call of a *groupby* method on our data set will return not a set of DataFrames, but a DataFrameGroupBy object:

In [ ]:
#Group by 'GeneralCountry' and create a DataFrameGroupBy object:
covid.groupby('GeneralCountry')

We can think of this object as of a special view of the DataFrame, which is already divided into groups. To get actual results, we need to apply an aggregate on this object:

In [ ]:
#For example, we can check when the date of latest observation for each country:
covid.groupby('GeneralCountry').ObservationDate.max().head()

In [ ]:
#Let's fix this ('ST. MARTIN') business:
covid['GeneralCountry'] = covid.GeneralCountry.str.strip("('")
covid.groupby('GeneralCountry').ObservationDate.max().head()

Similarly, we can use any of the summary functions with groupby object. For example, we can see latest numbers of confirmed cases within various regions of China:

In [ ]:
covid[covid.GeneralCountry == 'CHINA'].groupby('Region').Confirmed.max()

We can also group by more than one column:

In [ ]:
#Group by 'GeneralCountry' and 'Region' and see maximum (=total) number of 'Deaths':
covid.groupby(['GeneralCountry', 'Region']).Deaths.max().head()

In [ ]:
#Number of regions within each country:
covid.groupby('GeneralCountry').Region.nunique().head()

In [ ]:
#See 'Confirmed' and 'Deaths' numbers by GeneralCountry:
covid.groupby('GeneralCountry')[['Confirmed', 'Deaths']].max().head(10)

**reset_index()** method allows to reset index:

In [ ]:
#Reset index in just created dataframe:
covid.groupby('GeneralCountry')[['Confirmed', 'Deaths']].max().reset_index().head(10)

We can also use the `agg()` method to perform more general aggregations subsets of columns of our dataset. 

In [ ]:
covid.groupby('Country').agg(
    {
        'Deaths':'count', 
        'ObservationDate': ['min','max']
     
    }
).sort_values(('ObservationDate','min'))


### Joining data

Often enough, while analyzing data, we need to combine different DataFrames and/or Series in non-trivial ways. Pandas has a few neat methods for doing this. 

In cases when we have data in different DataFrames (or Series) but having the same fields (columns), **concat()** function lets us join dataframes along axis.

Let's say we create a new DataFrame using *groupby()* method:

In [ ]:
countries_stats = covid.groupby('GeneralCountry')[['Confirmed', 'Deaths']].max().reset_index()
countries_stats.head()

And pretend we have just heard of two new confirmed cases - one in Mongolia, and one in Jamaica:

In [ ]:
new_cases = pd.DataFrame({'GeneralCountry': ['MONGOLIA', 'JAMAICA'], 'Confirmed': [1.0, 1.0], 'Deaths': [0, 0]})
new_cases

Now if we would like to study all of the affected countries simultaneously, we would use **concat()** method to stack these two DataFrames on top of one another:

In [ ]:
#Concatenate counties_stats and new_cases in new 'all_cases' DataFrame:
all_cases = pd.concat([countries_stats, new_cases])
all_cases.head()

In [ ]:
#Check:
all_cases[all_cases.GeneralCountry == 'JAMAICA']

To concat the same two DataFrames along columns, we would switch *axis* parameter to 1:

In [ ]:
#Concatenate the same DataFrames along  columns:
all_cases_too = pd.concat([countries_stats, new_cases], axis = 1)
all_cases_too.head()

**merge()** or **join()** methods let you combine different DataFrame objects which have an index in common (similar to a JOIN in SQL). Th **join** method works the best when we need to join dataframes on their indexes, while the **merge** is more versatile.

Let's create a new DataFrame listing countries and latest observation dates:

In [ ]:
#Create a new DataFrame with countries and latest observation dates:
latest_data = covid.groupby(['GeneralCountry', 'Region']).ObservationDate.max().reset_index()
latest_data.head()

Now we want to join this DataFrame with our original 'covid' DataFrame to get latest fatality rates:

In [ ]:
latest_rates = pd.merge(latest_data, covid[['GeneralCountry', 'Region', 'ObservationDate', 'MortalityRate']], \
                        on=['GeneralCountry', 'Region', 'ObservationDate'])
latest_rates

We can also specify if we need inner, outer, right, or left join, as well as add a suffix to duplicate column names:

**pd.merge(df_1, df_2, on='ID', how='left', suffixes=('_1', '_2'))**

### Pivot tables

With pandas, we can also create a spreadsheet-style pivot table as a DataFrame:

In [ ]:
#Create an Excel-style pivot table:
pd.pivot_table(covid[['GeneralCountry', 'Region', 'ObservationDate', 'Confirmed']], \
               index=['GeneralCountry','Region'], aggfunc='max')

### Data visualizations with pandas

When we want to quickly view how our data looks, pandas visualizations come in handy.

Let's look again at *all_cases* DataFrame we created some time ago:

In [ ]:
#Look at first 5 rows of 'all_cases' DataFrame:
all_cases.head()

Pandas visualizations are built on top of famous **matplotlib** library.
The standard convention for referencing the matplotlib API:

In [ ]:
import matplotlib.pyplot as plt

To make sure all plots are visible within our notebook, use this magic command:

**%matplotlib inline**


In [ ]:
%matplotlib inline

To build charts with pandas, we use basic **plot()** method:

In [ ]:
#Let's look at all confirmed cases by date:
by_date = covid.groupby('ObservationDate').Confirmed.sum().reset_index()
by_date.head(2)

In [ ]:
#Now let's visualize this with simple line chart:
by_date.plot.line(x = 'ObservationDate', y = 'Confirmed', color = 'purple', title = 'COVID-19 confirmed cases: World')

Let us now build a similar plot for US only:

In [ ]:
#Create a new 'us_daily' DataFrame with US-only data:
us_daily = covid[covid.GeneralCountry == 'US'].groupby('ObservationDate').sum().reset_index()
us_daily

In [ ]:
#create a line chart:
us_daily.plot.line(x = 'ObservationDate', y = 'Confirmed', color = 'brown', title = 'COVID-19 confirmed cases: US')

Let's look at total number of cases and deaths by country:

In [ ]:
#First, let's group everything by GeneralCountry & Region and get latest numbers of confirmed cases and deaths:
world_regions = covid.groupby(['GeneralCountry', 'Region'])[['Confirmed', 'Deaths']].max().reset_index()
world_regions.head()

In [ ]:
#Then we can get total latest numbers for each country:
world = world_regions.groupby('GeneralCountry')[['Confirmed', 'Deaths']].sum().reset_index()
world.head()

In [ ]:
#One of the basic and widely used plots - bar chart:
world.plot(kind = 'bar')

Let's try to make it looking a bit more appealing by visualizing top 10 countries only, and make horizontal bar chart, instead:

In [ ]:
#Create a DataFrame with top 10 COVID-19 confirmed cases:
top = world.sort_values(by = 'Confirmed', ascending = False).head(10)
top

A little trick to make your dataframe look more presentable by adding background color (using matplotlib built-in colormaps (**cmap**)):

In [ ]:
top10 = top.style.background_gradient(cmap='Blues',subset=["Confirmed"]).background_gradient(cmap='Reds',subset=["Deaths"])
top10

Now let's plot the simplest bar chart:

In [ ]:
#Plot the bar chart with 'top' data:
top.plot.bar()

#another way of getting the same result is:
#top.plot(kind = 'bar')

Here, by default, **plot()** method built index values on x-axis. Let's specify that we want to see countries names, instead:

In [ ]:
top.plot.bar(x = 'GeneralCountry')

In [ ]:
#Convert this to horizontal bar chart:
top.index = top.GeneralCountry
top.plot.barh()

In [ ]:
#Modify some more:
top.index = top.GeneralCountry
top.plot.barh(title = 'Top 10 Countries (Confirmed Cases and Deaths)', figsize = [10,7],\
             fontsize = 12, color = ['blue', 'red']).invert_yaxis()

### Plotting with Plotly

The plotly Python library is an interactive, open-source plotting library that supports over 40 unique chart types covering a wide range of statistical, financial, geographic, scientific, and 3-dimensional use-cases.

In [ ]:
#Import plotly and check its version:
import plotly
plotly.__version__

In case you do not have plotly installed, run the following line in your Anaconda prompt:

**conda install -c plotly plotly=4.6.0**

We are going to use Plotly Express, which is easy-to-use, high-level interface to Plotly, which produces easy-to-style figures. Every Plotly Express function returns a graph_objects.Figure object whose data and layout has been pre-populated according to the provided arguments.

Note: Plotly Express was previously its own separately-installed **plotly_express** package but is now part of plotly and importable via 

**import plotly.express as px**

In [ ]:
import plotly.express as px
import numpy as np
temp_df = pd.DataFrame(world)
temp_df = temp_df.reset_index()
fig = px.choropleth(temp_df, locations="GeneralCountry",
                    color=np.log10(temp_df["Confirmed"]), 
                    hover_name="index", # column to add to hover information
                    hover_data=["Confirmed"],
                    color_continuous_scale=px.colors.sequential.Plasma,locationmode="country names")
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text="Confirmed Cases Heat Map (Log Scale)")
fig.update_coloraxes(colorbar_title="Confirmed Cases(Log Scale)",colorscale="Blues")
fig.show()

In [ ]:
#Another example visualizing progression of COVID-19 spread across the continents:
df_data = covid.groupby(['ObservationDate', 'GeneralCountry'])['Confirmed', 'Deaths'].max().reset_index().fillna(0)
df_data["ObservationDate"] = pd.to_datetime( df_data["ObservationDate"]).dt.strftime('%m/%d/%Y')
fig = px.scatter_geo(df_data, locations="GeneralCountry", locationmode='country names', 
                     color=np.power(df_data["Confirmed"],0.3)-2 , size= np.power(df_data["Confirmed"]+1,0.3)-1, hover_name="GeneralCountry",
                     hover_data=["Confirmed"],
                     range_color= [0, max(np.power(world["Confirmed"],0.3))], 
                     projection="natural earth", animation_frame="ObservationDate", 
                     color_continuous_scale=px.colors.sequential.Plasma,
                     title='COVID-19: Progression of spread'
                    )
fig.update_coloraxes(colorscale="hot")
fig.update(layout_coloraxis_showscale=False)
fig.show()